In [13]:
import pandas as pd
df = pd.read_csv('retail_store_inventory.csv')

## Handling the Missing Data 
### Filling the categorical Columns using mode and filling the numeric columns using mean

In [14]:
df['Region'] = df['Region'].fillna(df['Region'].mode()[0])
df['Weather Condition'] = df['Weather Condition'].fillna(df['Weather Condition'].mode()[0])
df['Holiday/Promotion'] = df['Holiday/Promotion'].fillna(df['Holiday/Promotion'].mode()[0])
df['Seasonality'] = df['Seasonality'].fillna(df['Seasonality'].mode()[0])
df['Units Ordered'] = df['Units Ordered'].fillna(df['Demand Forecast'])
df['Units Ordered'] = df['Units Ordered'].fillna(df.groupby('Category')['Units Ordered'].transform('mean'))
df['Units Ordered'] = df['Units Ordered'].fillna(0)

## Checking the missing values

In [15]:
df.isnull().sum()

Date                  0
Store ID              0
Product ID            0
Category              0
Region                0
Inventory Level       0
Units Sold            0
Units Ordered         0
Demand Forecast       0
Price                 0
Discount              0
Weather Condition     0
Holiday/Promotion     0
Competitor Pricing    0
Seasonality           0
dtype: int64

In [16]:
df.head(10)

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
0,1/1/2022,S001,P0001,Groceries,North,231,127,55.00,135.47,33.50,20,Rainy,0.0,29.69,Autumn
1,1/1/2022,S001,P0002,Toys,South,204,150,66.00,144.04,63.01,20,Sunny,0.0,66.16,Autumn
2,1/1/2022,S001,P0003,Toys,West,102,65,51.00,74.02,27.99,10,Sunny,1.0,31.32,Summer
3,1/1/2022,S001,P0004,Toys,North,469,61,62.18,62.18,32.72,10,Cloudy,1.0,34.74,Autumn
4,1/1/2022,S001,P0005,Electronics,East,166,14,135.00,9.26,73.64,0,Sunny,0.0,68.95,Summer
5,1/1/2022,S001,P0006,Groceries,South,138,128,102.00,139.82,76.83,10,Sunny,1.0,79.35,Winter
6,1/1/2022,S001,P0007,Furniture,East,359,97,167.00,108.92,34.16,10,Rainy,1.0,36.55,Winter
7,1/1/2022,S001,P0008,Clothing,North,380,312,54.00,329.73,97.99,5,Sunny,0.0,100.09,Spring
8,1/1/2022,S001,P0009,Electronics,West,183,175,135.00,174.15,20.74,10,Cloudy,0.0,17.66,Autumn
9,1/1/2022,S001,P0010,Toys,South,108,28,196.00,24.47,59.99,0,Rainy,1.0,61.21,Winter


## Navigating the total Units that have been sold during months

In [18]:
df['Month'] = df['Date'].dt.month
df['Month'] = df['Date'].dt.to_period('M')
#This line extracts the year-month period from each date and stores it in a new 'Month' column.
#This simplifies time-based analysis and makes it easier to group or summarize data by month.
df.groupby('Month')['Units Sold'].count()

Month
2022-01    3100
2022-02    2800
2022-03    3100
2022-04    3000
2022-05    3100
2022-06    3000
2022-07    3100
2022-08    3100
2022-09    3000
2022-10    3100
2022-11    3000
2022-12    3100
2023-01    3100
2023-02    2800
2023-03    3100
2023-04    3000
2023-05    3100
2023-06    3000
2023-07    3100
2023-08    3100
2023-09    3000
2023-10    3100
2023-11    3000
2023-12    3100
2024-01     100
Freq: M, Name: Units Sold, dtype: int64

## In this step, we identify and analyze the frequency of zero-sales cases across different months.
### This can help detect seasonal dips, stock issues, or product mismatch in specific periods.
#### Sorting by month helps visualize this chronologically.

In [20]:
zero_sales = df[df['Units Sold'] == 0]
print(zero_sales['Month'].value_counts().sort_index())


Month
2022-01    12
2022-02    10
2022-03    14
2022-04    12
2022-05    15
2022-06    15
2022-07    21
2022-08    12
2022-09    16
2022-10    14
2022-11    10
2022-12    17
2023-01    15
2023-02    17
2023-03    21
2023-04    14
2023-05    14
2023-06    26
2023-07    18
2023-08    19
2023-09    15
2023-10     6
2023-11    12
2023-12    15
Freq: M, Name: count, dtype: int64


#### In this step, we identify which products recorded zero units sold by counting how often each product appears in the filtered zero-sales data.

In [21]:
print(zero_sales['Product ID'].value_counts())


Product ID
P0019    25
P0004    24
P0018    24
P0015    24
P0010    24
P0020    20
P0007    20
P0014    20
P0017    19
P0001    18
P0008    16
P0016    16
P0006    16
P0013    16
P0011    15
P0002    15
P0003    13
P0012    13
P0005    12
P0009    10
Name: count, dtype: int64


## This step creates a comprehensive table that shows:

### The total occurrences of each Weather Condition for each month.

### The number of holidays (where Holiday/Promotion = 1) for each month.

## The table merges two groups:

### Weather counts: How often each weather condition occurs in each month.

### Holiday counts: The total number of holidays (sum of 1s for each month).

#### Note: If Holiday/Promotion = 0, it’s treated as a normal working day.

In [22]:
# Group by Month and Weather Condition to count occurrences
weather_counts = zero_sales.groupby(['Month', 'Weather Condition']).size().reset_index(name='Weather Condition Count')

# Group by Month to count the number of holidays (Holiday/Promotion = 1)
holiday_counts = zero_sales.groupby('Month')['Holiday/Promotion'].sum().reset_index(name='Holiday Count')

# Merge the two dataframes on 'Month' to display both counts together
full_table = pd.merge(weather_counts, holiday_counts, on='Month', how='left')

# Display the resulting full table
import pandas as pd
pd.set_option('display.max_rows', None)  # Optional: To display all rows without truncation
display(full_table)

,Month,Weather Condition,Weather Condition Count,Holiday Count
0,2022-01,Cloudy,2,6.0
1,2022-01,Rainy,2,6.0
2,2022-01,Snowy,2,6.0
3,2022-01,Sunny,6,6.0
4,2022-02,Cloudy,1,5.0
5,2022-02,Rainy,1,5.0
6,2022-02,Snowy,4,5.0
7,2022-02,Sunny,4,5.0
8,2022-03,Cloudy,2,5.0
9,2022-03,Rainy,3,5.0


In [23]:
df.head(10)


,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,Month
0,2022-01-01,S001,P0001,Groceries,North,231,127,55.00,135.47,33.50,20,Rainy,0.0,29.69,Autumn,2022-01
1,2022-01-01,S001,P0002,Toys,South,204,150,66.00,144.04,63.01,20,Sunny,0.0,66.16,Autumn,2022-01
2,2022-01-01,S001,P0003,Toys,West,102,65,51.00,74.02,27.99,10,Sunny,1.0,31.32,Summer,2022-01
3,2022-01-01,S001,P0004,Toys,North,469,61,62.18,62.18,32.72,10,Cloudy,1.0,34.74,Autumn,2022-01
4,2022-01-01,S001,P0005,Electronics,East,166,14,135.00,9.26,73.64,0,Sunny,0.0,68.95,Summer,2022-01
5,2022-01-01,S001,P0006,Groceries,South,138,128,102.00,139.82,76.83,10,Sunny,1.0,79.35,Winter,2022-01
6,2022-01-01,S001,P0007,Furniture,East,359,97,167.00,108.92,34.16,10,Rainy,1.0,36.55,Winter,2022-01
7,2022-01-01,S001,P0008,Clothing,North,380,312,54.00,329.73,97.99,5,Sunny,0.0,100.09,Spring,2022-01
8,2022-01-01,S001,P0009,Electronics,West,183,175,135.00,174.15,20.74,10,Cloudy,0.0,17.66,Autumn,2022-01
9,2022-01-01,S001,P0010,Toys,South,108,28,196.00,24.47,59.99,0,Rainy,1.0,61.21,Winter,2022-01


In [24]:
print("Cleaned dataset saved as 'CleanedDataSet.csv'")


Cleaned dataset saved as 'CleanedDataSet.csv'
